In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración

C:\Users\karin\AppData\Local\Temp\ipykernel_12844\1499216972.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Pair programming Nulos

## Ejercicios gestión de nulos

En este pair programming usaremos el csv que generamos en el pair programming de ayer. El objetivo del ejercicio es identificar las columnas que contienen valores nulos y aplicar un proceso de imputación para manejarlos de manera adecuada.

Instrucciones:

1. Identificación de Valores Nulos: Identifica todas las columnas que contengan valores nulos en el DataFrame.


In [2]:
# Cargamos el dataframe creado en la lección anterior
df = pd.read_csv("./df_world2.csv" ,index_col = 0)
df.head()

,country,density,abbreviation,agriculturalland,landarea,armedforcessize,birthrate,callingcode,capital/majorcity,co2-emissions,...,physiciansperthousand,population,population:laborforceparticipation,taxrevenue,totaltaxrate,unemploymentrate,urban_population,latitud,longitud,continent
0,Afghanistan,60.0,AF,58.1,652230.0,323000.0,32.49,93.0,Kabul,8672.0,...,0.28,38041754.0,48.9,9.3,71.4,11.12,9797273.0,33.939110,67.709953,Asia
1,Albania,105.0,AL,43.1,28748.0,9000.0,11.78,355.0,Tirana,4536.0,...,1.20,2854191.0,55.7,18.6,36.6,12.33,1747593.0,41.153332,20.168331,Europe
2,Algeria,18.0,DZ,17.4,2381741.0,317000.0,24.28,213.0,Algiers,150006.0,...,1.72,43053054.0,41.2,37.2,66.1,11.70,31510100.0,28.033886,1.659626,Africa
3,Andorra,164.0,AD,40.0,468.0,NaN,7.20,376.0,Andorra la Vella,469.0,...,3.33,77142.0,NaN,NaN,NaN,NaN,67873.0,42.506285,1.521801,Europe
4,Angola,26.0,AO,47.5,1246700.0,117000.0,40.73,244.0,Luanda,34693.0,...,0.21,31825295.0,77.5,9.2,49.1,6.89,21061025.0,-11.202692,17.873887,Africa


In [6]:
# Calculamos el porcentaje de nulos que tenemos para cada columna
(df.isnull().sum() / df.shape[0]) * 100

# Lo convertimos a DataFrame con porcentaje de nulos 
df_nulos = pd.DataFrame((df.isnull().sum() / df.shape[0]) * 100, columns = ["%_nulos"])

# filtramos el DataFrame para quedarnos solo con aquellas columnas que tengan nulos
df_nulos[df_nulos["%_nulos"] > 0]

,%_nulos
abbreviation,3.608247
agriculturalland,3.608247
landarea,0.515464
armedforcessize,12.371134
birthrate,3.092784
callingcode,0.515464
capital/majorcity,1.546392
co2-emissions,3.608247
cpi,8.762887
cpichange,8.247423


In [7]:
# Obtenemos la lista de columnas categóricas que tienen nulos: 
nulos_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['abbreviation', 'capital/majorcity', 'currency-code', 'largestcity',
       'officiallanguage'],
      dtype='object')


In [8]:
# sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulos_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df[col].value_counts() / df.shape[0])
    print("........................")

La distribución de las categorías para la columna ABBREVIATION


abbreviation
AF    0.005155
PG    0.005155
NI    0.005155
NE    0.005155
NG    0.005155
        ...   
GR    0.005155
GD    0.005155
GT    0.005155
GN    0.005155
ZW    0.005155
Name: count, Length: 187, dtype: float64

........................
La distribución de las categorías para la columna CAPITAL/MAJORCITY


capital/majorcity
Kabul                    0.005155
Ngerulmud                0.005155
Wellington               0.005155
Managua                  0.005155
Niamey                   0.005155
                           ...   
Athens                   0.005155
St. George's, Grenada    0.005155
Guatemala City           0.005155
Conakry                  0.005155
Harare                   0.005155
Name: count, Length: 191, dtype: float64

........................
La distribución de las categorías para la columna CURRENCY-CODE


...

........................
La distribución de las categorías para la columna LARGESTCITY


largestcity
Kabul                    0.005155
Panama City              0.005155
Managua                  0.005155
Niamey                   0.005155
Lagos                    0.005155
                           ...   
Accra                    0.005155
Macedonia                0.005155
St. George's, Grenada    0.005155
Guatemala City           0.005155
Harare                   0.005155
Name: count, Length: 188, dtype: float64

........................
La distribución de las categorías para la columna OFFICIALLANGUAGE


officiallanguage
English             0.159794
French              0.128866
Spanish             0.097938
Arabic              0.092784
Portuguese          0.036082
                      ...   
Albanian            0.005155
Jamaican English    0.005155
Lao                 0.005155
Latvian             0.005155
Shona               0.005155
Name: count, Length: 76, dtype: float64

........................


In [9]:
# Obtenemos la lista de columnas numéricas que tienen nulos
nulos_num = df[df.columns[df.isnull().any()]].select_dtypes(include = np.number).columns
print("Las columnas numéricas que tienen nulos son : \n ")
print(nulos_num)

Las columnas numéricas que tienen nulos son : 
 
Index(['agriculturalland', 'landarea', 'armedforcessize', 'birthrate',
       'callingcode', 'co2-emissions', 'cpi', 'cpichange', 'fertilityrate',
       'forestedarea', 'gasolineprice', 'gdp',
       'grossprimaryeducationenrollment', 'grosstertiaryeducationenrollment',
       'infantmortality', 'lifeexpectancy', 'maternalmortalityratio',
       'minimumwage', 'outofpockethealthexpenditure', 'physiciansperthousand',
       'population', 'population:laborforceparticipation', 'taxrevenue',
       'totaltaxrate', 'unemploymentrate', 'urban_population'],
      dtype='object')


In [10]:
# lo primero que vamos a hacer es calcular el número de nulos que tenemos en cada una de las columnas numericas
df[nulos_num].isnull().sum() / df.shape[0]

agriculturalland                      0.036082
landarea                              0.005155
armedforcessize                       0.123711
birthrate                             0.030928
callingcode                           0.005155
co2-emissions                         0.036082
cpi                                   0.087629
cpichange                             0.082474
fertilityrate                         0.036082
forestedarea                          0.036082
gasolineprice                         0.097938
gdp                                   0.010309
grossprimaryeducationenrollment       0.036082
grosstertiaryeducationenrollment      0.061856
infantmortality                       0.030928
lifeexpectancy                        0.041237
maternalmortalityratio                0.072165
minimumwage                           0.226804
outofpockethealthexpenditure          0.036082
physiciansperthousand                 0.036082
population                            0.005155
population:la


2. Selección de Método de Imputación: Discute en tu compañera cuál sería la mejor estrategia para manejar los valores nulos en cada una de las columnas identificadas en el paso anterior.



3. Imputación de Valores Nulos: Implementa el método de imputación seleccionado en el paso 2 para llenar los valores nulos en las columnas.



4. Informe: Añade al final de un jupyter una explicación breve que describa las columnas que tenían valores nulos, cómo decidiste imputarlos y cualquier observación adicional que consideres importante sobre el proceso de limpieza de datos.
